In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

In [ ]:
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_rows",10)
pd.set_option("display.max_columns",5)
DS= pd.read_pickle('Balanced_Math.pkl');
answer_data = DS
answer_data

In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key='your api key',
)

In [ ]:
def get_completion(messages, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo-0125"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:

import time

def connect_with_api(messages):
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            response = get_completion(messages)
            return response
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            print(f"Retrying... (Attempt {retries} of {max_retries})")
            time.sleep(1)  # Wait for 1 second before retrying

    print("Failed to connect with the API after multiple attempts.")

In [ ]:
# test_samples_per_answer = 199
# train_samples_per_answer = 1
test_samples_per_answer = 49
train_samples_per_answer = 1
test_df = pd.DataFrame()
train_df = pd.DataFrame()
for answer in answer_data['answer'].unique():
    answer_df = answer_data[answer_data['answer'] == answer]
    test_samples = answer_df.sample(test_samples_per_answer, random_state=1)
    train_samples = answer_df.drop(test_samples.index).sample(train_samples_per_answer, random_state=1)

    test_df = pd.concat([test_df, test_samples])
    train_df = pd.concat([train_df, train_samples])

# Reset the index of the resulting DataFrames
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

In [ ]:
import re

def clean_response(response):
    # Strip leading and trailing whitespace
    response = response.strip()

    # Define a pattern to capture the answer choice
    match = re.search(r'\b([abcde])\b', response)

    if match:
        return match.group(1)
    else:
        return None

In [ ]:

# Experiment configurations
def run_experiment(prompt_design, test_df, train_df, experiment_name):
    test_results = []
    for index, row in test_df.iterrows():
        if prompt_design == "zeroU":
            messages = [{"role": "user", "content": f"""Determine the answer of the example based on the provided question:
             For the question provided, classify its answer as a single small letter (without other marks or words like 'answer:'), either "a", "b", "c", "d", or "e".
```{row['question']}```""" }]
        elif prompt_design == "zeroSU":
            messages = [{"role": "system", "content": f"""Determine the answer of the example based on the provided question.
             For the question provided, classify its answer as a single small letter (without other marks or words like 'answer:'), either "a", "b", "c", "d", or "e"."""},
                        {"role": "user", "content": f"""```{row['question']}```"""}]
        elif prompt_design == "fewU":
            messages = [{"role": "user", "content": f"""Determine the answer of the example based on the provided question:
             For the question provided, classify its answer as a single small letter (without other marks or words like 'answer:'), either "a", "b", "c", "d", or "e".
Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']}
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']}
```{train_df.iloc[3]['question']}``` - {train_df.iloc[3]['answer']}
...
```{row['question']}```""" }]
        elif prompt_design == "fewSU":
            messages = [{"role": "system", "content": f"""Determine the answer of the example based on the provided question.
             For the question provided, classify its answer as a single small letter (without other marks or words like 'answer:'), either "a", "b", "c", "d", or "e"."""},
                        {"role": "user", "content": f"""Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']}
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']}
```{train_df.iloc[3]['question']}``` - {train_df.iloc[3]['answer']}
...
```{row['question']}```"""}]
        elif prompt_design == "fewSUA":
            messages = [{"role": "system", "content": f"""Determine the answer of the example based on the provided question.
            For the question provided, classify its answer as a single small letter (without other marks or words like 'answer:'), either "a", "b", "c", "d", or "e"."""},
                        {"role": "user", "content": train_df.iloc[0]['question']}, {"role": "assistant", "content": train_df.iloc[0]['answer']},
                        {"role": "user", "content": train_df.iloc[2]['question']}, {"role": "assistant", "content": train_df.iloc[2]['answer']},
                        {"role": "user", "content": train_df.iloc[3]['question']}, {"role": "assistant", "content": train_df.iloc[3]['answer']},
                        {"role": "user", "content": f"""```{row['question']}```"""}]

        prediction = connect_with_api(messages)
        print(index, ' ', prediction)

        test_results.append({'Index': index, 'question': row['question'], 'Predicted_answer': prediction, 'Actual_answer': row['answer']})

    results_df = pd.DataFrame(test_results)
    # Calculate structural accuracy before cleaning
    valid_answer = ['A', 'B', 'C', 'D','E']
    results_df['Is_Structurally_Valid'] = results_df['Predicted_answer'].apply(lambda x: x.strip().upper() in valid_answer)
    structural_accuracy = results_df['Is_Structurally_Valid'].mean()

    # Calculate F1 Score
    results_df['Predicted_answer'] = results_df['Predicted_answer'].apply(clean_response)
    results_df = results_df.dropna(subset=['Predicted_answer', 'Actual_answer'])
    f1 = f1_score(results_df['Actual_answer'], results_df['Predicted_answer'], average='micro')

    print(f"Experiment: {experiment_name}")
    print(f"The F1 Score for the predicted answer is: {f1}")
    print(f"The Structural Accuracy is: {structural_accuracy}")

    results_df.to_csv(f"{experiment_name}_results.csv", index=False)
    print(f"Results for {experiment_name} saved to {experiment_name}_results.csv")

In [ ]:
# Running the experiments
run_experiment("zeroU", test_df, train_df, "Zero-shot User Prompt")

run_experiment("zeroSU", test_df, train_df, "Zero-shot System and User Prompt")

run_experiment("fewU", test_df, train_df, "Few-shot User Prompt")

run_experiment("fewSU", test_df, train_df, "Few-shot System and User Prompt")

run_experiment("fewSUA", test_df, train_df, "Few-shot System, User, and Assistant Prompt")

In [ ]:

# Experiment configurations
def run_experiment(prompt_design, test_df, train_df, experiment_name):
    test_results = []
    for index, row in test_df.iterrows():
        if prompt_design == "zeroU":
            messages = [{"role": "user", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single samall letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e".
```{row['question']}```""" }]
        elif prompt_design == "zeroSU":
            messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single small letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e"."""},
                        {"role": "user", "content": f"""```{row['question']}```"""}]
        elif prompt_design == "fewU":
            messages = [{"role": "user", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single samall letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e".
Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']}
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']}
```{train_df.iloc[3]['question']}``` - {train_df.iloc[3]['answer']}
...
```{row['question']}```""" }]
        elif prompt_design == "fewSU":
            messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single small letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e"."""},
                        {"role": "user", "content": f"""Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']}
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']}
```{train_df.iloc[3]['question']}``` - {train_df.iloc[3]['answer']}
...
```{row['question']}```"""}]
        elif prompt_design == "fewSUA":
            messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single small letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e"."""},
                        {"role": "user", "content": train_df.iloc[0]['question']}, {"role": "assistant", "content": train_df.iloc[0]['answer']},
                        {"role": "user", "content": train_df.iloc[2]['question']}, {"role": "assistant", "content": train_df.iloc[2]['answer']},
                        {"role": "user", "content": train_df.iloc[3]['question']}, {"role": "assistant", "content": train_df.iloc[3]['answer']},
                        {"role": "user", "content": f"""```{row['question']}```"""}]

        prediction = connect_with_api(messages)
        print(index, ' ', prediction)

        test_results.append({'Index': index, 'question': row['question'], 'Predicted_answer': prediction, 'Actual_answer': row['answer']})

    results_df = pd.DataFrame(test_results)
    # Calculate structural accuracy before cleaning
    valid_answer = ['A', 'B', 'C', 'D','E']
    results_df['Is_Structurally_Valid'] = results_df['Predicted_answer'].apply(lambda x: x.strip().upper() in valid_answer)
    structural_accuracy = results_df['Is_Structurally_Valid'].mean()

    # Calculate F1 Score
    results_df['Predicted_answer'] = results_df['Predicted_answer'].apply(clean_response)
    results_df = results_df.dropna(subset=['Predicted_answer', 'Actual_answer'])
    f1 = f1_score(results_df['Actual_answer'], results_df['Predicted_answer'], average='micro')

    print(f"Experiment: {experiment_name}")
    print(f"The F1 Score for the predicted answer is: {f1}")
    print(f"The Structural Accuracy is: {structural_accuracy}")

    results_df.to_csv(f"{experiment_name}_results.csv", index=False)
    print(f"Results for {experiment_name} saved to {experiment_name}_results.csv")

In [ ]:
# Running the experiments
run_experiment("zeroU", test_df, train_df, "Zero-shot User Prompt")

run_experiment("zeroSU", test_df, train_df, "Zero-shot System and User Prompt")

run_experiment("fewU", test_df, train_df, "Few-shot User Prompt")

run_experiment("fewSU", test_df, train_df, "Few-shot System and User Prompt")

run_experiment("fewSUA", test_df, train_df, "Few-shot System, User, and Assistant Prompt")

In [ ]:
import re
# Experiment configurations
def run_experiment(prompt_design, test_df, train_df, experiment_name):
    test_results = []
    for index, row in test_df.iterrows():
        if prompt_design == "zeroU":
            messages = [{"role": "user", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single samall letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e".
                 After providing the letter answer, explain your rationale and how you arrived at the solution.
```{row['question']}```""" }]
        elif prompt_design == "zeroSU":
            messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single small letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e".
                 After providing the letter answer, explain your rationale and how you arrived at the solution."""},
                        {"role": "user", "content": f"""```{row['question']}```"""}]
        elif prompt_design == "fewU":
            messages = [{"role": "user", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single samall letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e".
                 After providing the letter answer, explain your rationale and how you arrived at the solution.
Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']} Explanation : the percent of alcohol in the solution is ( 0.05 ( 40 ) + 2.5 ) / 50 = 4.5 / 50 = 9 % the answer is a
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']} Explanation : total = 100 t = 80 nt = 20 80 * ( 20 / 100 ) = 24 80 * ( 20 / 100 ) = 24 16 + 16 = 32 = > 100 - 32 = 68 % answer : b
```{train_df.iloc[3]['question']}``` - {train_df.iloc[3]['answer']} Explanation : let the price be = rs . 100 , and number of units sold = 100 then , sale value = rs . ( 100 \u00d7 100 ) = rs . 10000 new sale value = rs . ( 170 \u00d7 80 ) = rs . 13600 increase % = 3600 / 10000 \u00d7 100 = 36 % answer : c
...
```{row['question']}```""" }]
        elif prompt_design == "fewSU":
            messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single small letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e".
                 After providing the letter answer, explain your rationale and how you arrived at the solution."""},
                        {"role": "user", "content": f"""Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']} Explanation : the percent of alcohol in the solution is ( 0.05 ( 40 ) + 2.5 ) / 50 = 4.5 / 50 = 9 % the answer is a
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']} Explanation : total = 100 t = 80 nt = 20 80 * ( 20 / 100 ) = 24 80 * ( 20 / 100 ) = 24 16 + 16 = 32 = > 100 - 32 = 68 % answer : b
```{train_df.iloc[3]['question']}``` - {train_df.iloc[3]['answer']} Explanation : let the price be = rs . 100 , and number of units sold = 100 then , sale value = rs . ( 100 \u00d7 100 ) = rs . 10000 new sale value = rs . ( 170 \u00d7 80 ) = rs . 13600 increase % = 3600 / 10000 \u00d7 100 = 36 % answer : c
...
```{row['question']}```"""}]
        elif prompt_design == "fewSUA":
            messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
                 Provide the answer as a single small letter (without any additional marks or words like 'answer:'), choosing from: "a", "b", "c", "d", or "e".
                 After providing the letter answer, explain your rationale and how you arrived at the solution."""},
                        {"role": "user", "content": train_df.iloc[0]['question']}, {"role": "assistant", "content": f""" {train_df.iloc[0]['answer']} Explanation : the percent of alcohol in the solution is ( 0.05 ( 40 ) + 2.5 ) / 50 = 4.5 / 50 = 9 % the answer is a"""},
                        {"role": "user", "content": train_df.iloc[2]['question']}, {"role": "assistant", "content": f""" {train_df.iloc[2]['answer']} Explanation : total = 100 t = 80 nt = 20 80 * ( 20 / 100 ) = 24 80 * ( 20 / 100 ) = 24 16 + 16 = 32 = > 100 - 32 = 68 % answer : b"""},
                        {"role": "user", "content": train_df.iloc[3]['question']}, {"role": "assistant", "content": f""" {train_df.iloc[3]['answer']} Explanation : let the price be = rs . 100 , and number of units sold = 100 then , sale value = rs . ( 100 \u00d7 100 ) = rs . 10000 new sale value = rs . ( 170 \u00d7 80 ) = rs . 13600 increase % = 3600 / 10000 \u00d7 100 = 36 % answer : c"""},
                        {"role": "user", "content": f"""```{row['question']}```"""}]


        prediction = connect_with_api(messages)
        print(index, ' ', prediction)

        test_results.append({'Index': index, 'question': row['question'], 'Predicted_answer': prediction, 'Actual_answer': row['answer']})

    results_df = pd.DataFrame(test_results)
    # Calculate structural accuracy before cleaning
    def check_structural_validity(response):
        # Check if the response starts with a single lowercase letter among a, b, c, d, e
        pattern = r'^[abcde](?!\w)'
        match = re.match(pattern, response.strip().lower())
        return bool(match)

    valid_answer = ['a', 'b', 'c', 'd', 'e']
    results_df['Is_Structurally_Valid'] = results_df['Predicted_answer'].apply(check_structural_validity)
    structural_accuracy = results_df['Is_Structurally_Valid'].mean()
    # Calculate F1 Score
    results_df['Predicted_answer'] = results_df['Predicted_answer'].apply(clean_response)
    results_df = results_df.dropna(subset=['Predicted_answer', 'Actual_answer'])
    f1 = f1_score(results_df['Actual_answer'], results_df['Predicted_answer'], average='micro')

    print(f"Experiment: {experiment_name}")
    print(f"The F1 Score for the predicted answer is: {f1}")
    print(f"The Structural Accuracy is: {structural_accuracy}")

    results_df.to_csv(f"{experiment_name}_results.csv", index=False)
    print(f"Results for {experiment_name} saved to {experiment_name}_results.csv")

In [ ]:
# Running the experiments
run_experiment("zeroU", test_df, train_df, "Zero-shot User Prompt")

run_experiment("zeroSU", test_df, train_df, "Zero-shot System and User Prompt")

run_experiment("fewU", test_df, train_df, "Few-shot User Prompt")

run_experiment("fewSU", test_df, train_df, "Few-shot System and User Prompt")

run_experiment("fewSUA", test_df, train_df, "Few-shot System, User, and Assistant Prompt")

In [ ]:
import re
# Experiment configurations
def run_experiment(prompt_design, test_df, train_df, experiment_name):
    test_results = []
    for index, row in test_df.iterrows():
        if prompt_design == "zeroU":
            messages = [{"role": "user", "content": f"""Solve the following math question and determine the correct answer.
               Explain your rationale and how you arrived at the solution.
               After explaining how reached the answer, Provide the answer as a single small letter (without any additional marks), choosing from: "a", "b", "c", "d", or "e".
```{row['question']}```""" }]
        elif prompt_design == "zeroSU":
            messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
               Explain your rationale and how you arrived at the solution.
               After explaining how reached the answer, Provide the answer as a single small letter (without any additional marks), choosing from: "a", "b", "c", "d", or "e"."""},
                        {"role": "user", "content": f"""```{row['question']}```"""}]
        elif prompt_design == "fewU":
            messages = [{"role": "user", "content": f"""Solve the following math question and determine the correct answer.
               Explain your rationale and how you arrived at the solution.
               After explaining how reached the answer, Provide the answer as a single small letter (without any additional marks), choosing from: "a", "b", "c", "d", or "e".
Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']} Explanation : the percent of alcohol in the solution is ( 0.05 ( 40 ) + 2.5 ) / 50 = 4.5 / 50 = 9 % the answer is a
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']} Explanation : total = 100 t = 80 nt = 20 80 * ( 20 / 100 ) = 24 80 * ( 20 / 100 ) = 24 16 + 16 = 32 = > 100 - 32 = 68 % answer : b
```{train_df.iloc[3]['question']}``` - {train_df.iloc[3]['answer']} Explanation : let the price be = rs . 100 , and number of units sold = 100 then , sale value = rs . ( 100 \u00d7 100 ) = rs . 10000 new sale value = rs . ( 170 \u00d7 80 ) = rs . 13600 increase % = 3600 / 10000 \u00d7 100 = 36 % answer : c
...
```{row['question']}```""" }]
        elif prompt_design == "fewSU":
            messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
               Explain your rationale and how you arrived at the solution.
               After explaining how reached the answer, Provide the answer as a single small letter (without any additional marks), choosing from: "a", "b", "c", "d", or "e"."""},
                        {"role": "user", "content": f"""Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']} Explanation : the percent of alcohol in the solution is ( 0.05 ( 40 ) + 2.5 ) / 50 = 4.5 / 50 = 9 % the answer is a
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']} Explanation : total = 100 t = 80 nt = 20 80 * ( 20 / 100 ) = 24 80 * ( 20 / 100 ) = 24 16 + 16 = 32 = > 100 - 32 = 68 % answer : b
```{train_df.iloc[3]['question']}``` - {train_df.iloc[3]['answer']} Explanation : let the price be = rs . 100 , and number of units sold = 100 then , sale value = rs . ( 100 \u00d7 100 ) = rs . 10000 new sale value = rs . ( 170 \u00d7 80 ) = rs . 13600 increase % = 3600 / 10000 \u00d7 100 = 36 % answer : c
...
```{row['question']}```"""}]

        elif prompt_design == "fewSUA":
           messages = [{"role": "system", "content": f"""Solve the following math question and determine the correct answer.
               Explain your rationale and how you arrived at the solution.
               After explaining how reached the answer, Provide the answer as a single small letter (without any additional marks), choosing from: "a", "b", "c", "d", or "e". """},
                {"role": "user", "content": train_df.iloc[0]['question']}, {"role": "assistant", "content": f""" Explanation : the percent of alcohol in the solution is ( 0.05 ( 40 ) + 2.5 ) / 50 = 4.5 / 50 = 9 % the answer is a"""},
                {"role": "user", "content": train_df.iloc[2]['question']}, {"role": "assistant", "content": f""" Explanation : total = 100 t = 80 nt = 20 80 * ( 20 / 100 ) = 24 80 * ( 20 / 100 ) = 24 16 + 16 = 32 = > 100 - 32 = 68 % answer : b"""},
                {"role": "user", "content": train_df.iloc[3]['question']}, {"role": "assistant", "content": f""" Explanation : let the price be = rs . 100 , and number of units sold = 100 then , sale value = rs . ( 100 \u00d7 100 ) = rs . 10000 new sale value = rs . ( 170 \u00d7 80 ) = rs . 13600 increase % = 3600 / 10000 \u00d7 100 = 36 % answer : c"""},
                {"role": "user", "content": f"""```{row['question']}```"""}]

        prediction = connect_with_api(messages)
        print(index, ' ', prediction)

        test_results.append({'Index': index, 'question': row['question'], 'Predicted_answer': prediction, 'Actual_answer': row['answer']})

    results_df = pd.DataFrame(test_results)
    # Calculate structural accuracy before cleaning
    def check_structural_validity(response):
        # Check if the response starts with a single lowercase letter among a, b, c, d, e
        pattern = r'^[abcde](?!\w)'
        match = re.match(pattern, response.strip().lower())
        return bool(match)

    valid_answer = ['a', 'b', 'c', 'd', 'e']
    results_df['Is_Structurally_Valid'] = results_df['Predicted_answer'].apply(check_structural_validity)
    structural_accuracy = results_df['Is_Structurally_Valid'].mean()
    # Calculate F1 Score
    results_df['Predicted_answer'] = results_df['Predicted_answer'].apply(clean_response)
    results_df = results_df.dropna(subset=['Predicted_answer', 'Actual_answer'])
    f1 = f1_score(results_df['Actual_answer'], results_df['Predicted_answer'], average='micro')

    print(f"Experiment: {experiment_name}")
    print(f"The F1 Score for the predicted answer is: {f1}")
    print(f"The Structural Accuracy is: {structural_accuracy}")

    results_df.to_csv(f"{experiment_name}_results.csv", index=False)
    print(f"Results for {experiment_name} saved to {experiment_name}_results.csv")

In [ ]:
# Running the experiments
run_experiment("zeroU", test_df, train_df, "Zero-shot User Prompt")

run_experiment("zeroSU", test_df, train_df, "Zero-shot System and User Prompt")

run_experiment("fewU", test_df, train_df, "Few-shot User Prompt")

run_experiment("fewSU", test_df, train_df, "Few-shot System and User Prompt")

run_experiment("fewSUA", test_df, train_df, "Few-shot System, User, and Assistant Prompt")